In [1]:
import pandas as pd
import numpy as np
from scripts import *

In [2]:
imdb1 = pd.read_csv("zippedData/imdb.name.basics.csv.gz")
imdb2 = pd.read_csv("zippedData/imdb.title.akas.csv.gz")
imdb3 = pd.read_csv("zippedData/imdb.title.basics.csv.gz")
imdb4 = pd.read_csv("zippedData/imdb.title.crew.csv.gz")
imdb5 = pd.read_csv("zippedData/imdb.title.principals.csv.gz")
imdb6 = pd.read_csv("zippedData/imdb.title.ratings.csv.gz")

# IMDB Web Scraped Data

In [3]:
# cleaning primary keys in imdb dfs
imdb2['title_id'] = imdb2['title_id'].str.strip()
imdb3['tconst'] = imdb3['tconst'].str.strip()
imdb4['tconst'] = imdb4['tconst'].str.strip()
imdb5['tconst'] = imdb5['tconst'].str.strip()
imdb6['tconst'] = imdb6['tconst'].str.strip()

In [4]:
# dropping duplicates
imdb2.drop_duplicates(subset=['title_id'], inplace=True)
imdb3.drop_duplicates(subset=['tconst'], inplace=True)
imdb4.drop_duplicates(subset=['tconst'], inplace=True)
imdb5.drop_duplicates(subset=['tconst'], inplace=True)
imdb6.drop_duplicates(subset=['tconst'], inplace=True)

In [5]:
# importing web scrapes
web_scrape1 = pd.read_csv('scraped_data/imdb_monetary_data_16590.csv')
web_scrape1.set_index(['title_id'], inplace=True)

In [6]:
web_scrape2 = pd.read_csv('scraped_data/imdb_monetary_data_16591_to_25214_.csv')
web_scrape2.set_index(['title_id'], inplace=True)

In [7]:
web_scrape3 = pd.read_csv('scraped_data/imdb_monetary_data_29214_.csv')
web_scrape3.set_index(['title_id'], inplace=True)

In [8]:
web_scrape4 = pd.read_csv('scraped_data/imdb_monetary_data_33736_.csv')
web_scrape4.set_index(['title_id'], inplace=True)

In [9]:
web_scrape5 = pd.read_csv('scraped_data/imdb_monetary_data_49857_.csv')
web_scrape5.set_index(['title_id'], inplace=True)

In [10]:
web_scrape6 = pd.read_csv('scraped_data/imdb_monetary_data_65930_.csv')
web_scrape6.set_index(['title_id'], inplace=True)

In [11]:
web_scrape7 = pd.read_csv('scraped_data/imdb_monetary_data_67456_.csv')
web_scrape7.set_index(['title_id'], inplace=True)

In [12]:
web_scrape8 = pd.read_csv('scraped_data/imdb_monetary_data_77436_.csv')
web_scrape8.set_index(['title_id'], inplace=True)

In [13]:
web_scrape9 = pd.read_csv('scraped_data/imdb_monetary_data_102849_.csv')
web_scrape9.set_index(['title_id'], inplace=True)

In [14]:
# concatenating webscraped data
web_scrape = pd.concat([web_scrape1, web_scrape2, web_scrape3, web_scrape4, web_scrape5, web_scrape6, web_scrape7, web_scrape8, web_scrape9])

In [15]:
# dropping unnamed column
web_scrape.drop(['Unnamed: 0'], axis=1, inplace=True)


In [16]:
# cleaning title_id (index)
web_scrape.index = web_scrape.index.str.strip()

In [17]:
# merging imdb2 - imdb6
df = web_scrape.merge(imdb2, left_on=web_scrape.index, right_on=['title_id'])

In [18]:
df = df.merge(imdb3, left_on=['title_id'], right_on=['tconst'])

In [19]:
df = df.merge(imdb4, left_on=['title_id'], right_on=['tconst'])

In [20]:
df = df.merge(imdb5, left_on=['title_id'], right_on=['tconst'])

In [21]:
df = df.merge(imdb6, how='left', left_on=['title_id'], right_on=['tconst'])

### Concatenating Director Tables

In [22]:
directors1 = pd.read_csv('scraped_data/imdb_name_list_202_.csv')

In [23]:
directors2 = pd.read_csv('scraped_data/imdb_name_list_304_.csv')

In [24]:
directors3 = pd.read_csv('scraped_data/imdb_name_list_3016_.csv')

In [25]:
directors4 = pd.read_csv('scraped_data/imdb_name_list_7799_.csv')

In [26]:
directors5 = pd.read_csv('scraped_data/imdb_name_list_13173_.csv')

In [27]:
directors_names = pd.concat([directors1, directors2, directors3, directors4, directors5])

### Merging Director Names with Df

In [28]:
# adding in scraped directors names
df = df.merge(directors_names, left_on=['directors'], right_on=['name_id'], how='left')

In [29]:
# dropping duplicates by unique title_id number
df.drop_duplicates(subset='title_id', inplace=True)

In [30]:
# dropping unnecessary columns
df.drop(['ordering_x', 'language', 'attributes', 'original_title', 'Runtime', 'is_original_title', 'tconst_x', 'tconst_y', 'types', 'category', 'job', 'characters', 'ordering_y', 'title', 'nconst', 'region', 'writers', 'directors', 'Unnamed: 0', 'name_id'], axis=1, inplace=True)

# Data Cleaning

In [31]:
# Dropping Null Values
df = df.dropna(subset=['Opening Weekend USA', 'Gross USA', 'Cumulative Worldwide Gross'], how='all')

In [32]:
# Cumulative Worldwide Gross replace NaN with 0
df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].fillna(0)

In [33]:
# dropping a few rows with strange string values
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('INR') == True].index)
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('NPR') == True].index)
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('AUD') == True].index)

In [34]:
# cleaning the currency strings to be usable integers
df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].map(lambda x: clean_currency(x))
df['Cumulative Worldwide Gross'] = pd.to_numeric(df['Cumulative Worldwide Gross'])

In [35]:
# Gross USA 
df['Gross USA'] = df['Gross USA'].fillna(0)
df['Gross USA'] = df['Gross USA'].map(lambda x: clean_currency(x))
df['Gross USA'] = pd.to_numeric(df['Gross USA'])

In [36]:
# Opening Weekend USA
df['Opening Weekend USA'] = df['Opening Weekend USA'].fillna(0)
df['Opening Weekend USA'] = df['Opening Weekend USA'].map(lambda x: clean_currency(x))
df['Opening Weekend USA'] = pd.to_numeric(df['Opening Weekend USA'])

In [37]:
# Budget
df['Budget'] = df['Budget'].map(lambda x: clean_currency(x))
df['Budget'] = df['Budget'].fillna('0')

In [38]:
# acquiring unique currencies to be able to convert foreign currencies to USD
budget_curr = df['Budget'].tolist()
list_of_curr = find_currencies(budget_curr).tolist()

## Switched null values to '0' string in order to apply conversion function

In [39]:
df.Budget.fillna('0', inplace = True)

## Apply conversion function

In [40]:
df['Budget'] = df['Budget'].apply(lambda x: conversion_rates(x))

In [41]:
df.shape

(17172, 13)

# Cleaning Genres Column

In [42]:
df['genres'] = df['genres'].fillna('NaN')

In [43]:
df['genres'] = df['genres'].map(lambda x: list(x.split(',')))

# Rename Director Column

In [44]:
df.rename(columns={'name':'director'}, inplace=True)

# Drop Budgets < 1000usd

In [45]:
df['Budget'] = df['Budget'].replace(0.0, np.nan)
df = df.drop(df[df.Budget < 1000].index)

# Make Profit and Roi Columns

In [46]:
df['Profit'] = df['Cumulative Worldwide Gross']-df['Budget']
df['Roi'] = ((df['Cumulative Worldwide Gross'] - df['Budget'])/df['Budget'])*100

In [47]:
df['Budget'] = df['Budget'].fillna(0)
df['Budget'] = df['Budget'].astype(int)
df['Profit'] = df['Profit'].fillna(0)
df['Profit'] = df['Profit'].astype(int)
df['Roi'] = df['Roi'].replace(np.inf, np.nan)

# Individual Genre Columns Added

In [48]:
# creating a list of unique genres found in our table
list_of_genres = [row[0] for row in df['genres']]
l_o_genres = list(set(list_of_genres))
l_o_genres.remove('NaN')

# creating a matrix of zeros that matches the length of our table and the width of the list of unique genres
zero_matrix = np.zeros((len(df), len(l_o_genres)))
dummies = pd.DataFrame(zero_matrix, columns=l_o_genres)

# adding 1s to each dummy cell whose genre is listed in the genres column
for i, gen in enumerate(df.genres):
    indices = dummies.columns.get_indexer(gen)
    dummies.iloc[i, indices] = 1

# merging the dummy matrix to our table
df = df.merge(dummies, how='outer', on=df.index)

# Reset Index

In [49]:
df.rename(columns={'key_0':'title_id'}, inplace=True)
df.set_index(['primary_title'], inplace=True)
df.drop(['title_id'], axis=1, inplace=True)

# Make df a .csv

In [50]:
df.to_csv('master_table.csv')